In [1]:
# Initialise variables and imports
import pyroomacoustics.doa as pra
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from room_simulation import Simulation
from sine_dataset import SineData
from network import SSLConvNet as ConvNet
from network import SSLConvNetCosLoss as ConvNetCosLoss
import matplotlib.pyplot as plt
import math

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
MODELS_PATH = "../models/"
DATA_PATH = "../thesis_data/"
BATCH_SIZE = 5
BATCH_SIZE_MSECOS = 25
TEST_SIZE = 10000
EPOCHS = int(TEST_SIZE/BATCH_SIZE)
EPOCHS_MSECOS = int(TEST_SIZE/BATCH_SIZE_MSECOS)
NR_MICS = 2
RAD = 50
RADII = [50, 500, 5000]
ROOM_SIMS = []
DATASETS_EXP2 = []
MIC_L_DIST = (11, -10)
MIC_R_DIST = (11, -10)
ABSORPTION = 0.0
MIN_FREQ = 20
MAX_FREQ = 20000
SAMPLE_RATE = int(MAX_FREQ*2.2)
TIME = 1
MIN_LENGTH = 65000
MIN_LENGTH_MSECOS = 48000
TO_RAD = np.pi/180
TO_DEG = 180/np.pi

# Define custom loss function
class CosBorderLoss(torch.nn.Module):

    def __init__(self):
        super(CosBorderLoss, self).__init__()

    def forward(self, pred, target):
        radial = torch.abs(torch.cos(pred-target) - torch.cos(target-target))
        border = nn.functional.relu(pred-2*np.pi) + nn.functional.relu(-pred)
        return torch.sum(radial + border)

def calcCircumPos(x1, y1, r, theta):
    xPositions = x1 + r*torch.sin(theta)
    xPositions = r*2-xPositions #Flip x-coordinates
    yPositions = y1 + r*(1-torch.cos(theta))
    yPositions = r*2-yPositions #Flip y-coordinates
    return xPositions, yPositions

def calcCircumAngle(cx, cy, x, y):
    theta = torch.atan2(cy-y, cx-x)
    theta = torch.remainder(torch.remainder(theta, 2*np.pi) + np.pi/2, 2*np.pi)
    return theta

def computeMeanITD(inL, inR):
    non_zero_ind_inL = np.argmax((inL > 0))
    non_zero_ind_inR = np.argmax((inR > 0))
    
    return (non_zero_ind_inL/SAMPLE_RATE - non_zero_ind_inR/SAMPLE_RATE) * 1000

def computeMeanIID(inL, inR):
    peak_inL_idxs = pra.detect_peaks(inL)
    peak_inR_idxs = pra.detect_peaks(inR)

    len_diff = len(peak_inL_idxs) - len(peak_inR_idxs)
    if len_diff != 0:
        if len_diff > 0:
            peak_inL_idxs = peak_inL_idxs[len_diff:]
        else:
            peak_inR_idxs = peak_inR_idxs[abs(len_diff):]
    
    return np.mean((inL[peak_inL_idxs] - inR[peak_inR_idxs]))

In [2]:
for rad in RADII:
    print(rad)
    roomSim = Simulation(SAMPLE_RATE, rad, ABSORPTION,MIC_L_DIST, MIC_R_DIST, NR_MICS)
    ROOM_SIMS.append(roomSim)

    DATASETS_EXP2.append(SineData(BATCH_SIZE, roomSim, TIME, MIN_LENGTH, MIN_FREQ, MAX_FREQ))

roomSimMSECos = ROOM_SIMS[0]
datasetMSECos = SineData(BATCH_SIZE_MSECOS, roomSimMSECos, TIME, MIN_LENGTH_MSECOS, MIN_FREQ, MAX_FREQ)

roomSim = Simulation(SAMPLE_RATE, 50, ABSORPTION,MIC_L_DIST, MIC_R_DIST, NR_MICS)
datasetLowFreq = SineData(1, roomSim, TIME, MIN_LENGTH_MSECOS, 20, 1500)
datasetMedFreq = SineData(1, roomSim, TIME, MIN_LENGTH_MSECOS, 1500, 3000)
datasetHigFreq = SineData(1, roomSim, TIME, MIN_LENGTH_MSECOS, 3000, 20000)
DATASETS_EXP3 = [datasetLowFreq, datasetMedFreq, datasetHigFreq]

50
500
5000


In [3]:
# Create networks
nets = []
for rad in RADII:
    print(rad)
    net = ConvNet(MIN_LENGTH).double()
    net.load_state_dict(torch.load(MODELS_PATH+"ConvNet_Rad"+str(rad), map_location="cpu"))
    net.eval()
    nets.append(net)

MSENet = ConvNet(MIN_LENGTH_MSECOS).double()
MSENet.load_state_dict(torch.load(MODELS_PATH+"ConvNet_MSELoss", map_location="cpu"))
MSENet.eval()

CosNet = ConvNetCosLoss(MIN_LENGTH_MSECOS).double()
CosNet.load_state_dict(torch.load(MODELS_PATH+"ConvNet_CosLoss", map_location="cpu"))
CosNet.eval()


50


FileNotFoundError: [Errno 2] No such file or directory: '../models/ConvNet_Rad50'

In [ ]:
################
# EXPERIMENT 1 #
################

CosLoss = CosBorderLoss()
MSELoss = nn.MSELoss()
CosLosses = np.zeros((2, EPOCHS_MSECOS))
MSELosses = np.zeros((2, EPOCHS_MSECOS))
dataLoader = DataLoader(datasetMSECos, batch_size=BATCH_SIZE_MSECOS)

for i in range(EPOCHS_MSECOS):
    if i%100 == 0:
        print("Epoch "+str(i)+"/"+str(EPOCHS_MSECOS))
    
    # Retrieve all input data and labels
    inL, inR, labelX, labelY, labelAzi = next(iter(dataLoader))
    inL = inL.double().to(device)
    inR = inR.double().to(device)
    labelX = labelX.double().to(device)
    labelY = labelY.double().to(device)
    labelAzi = labelAzi.double().to(device)

    # Compute MSE and Cos losses for coordinate network
    MSENet = MSENet.to(device)
    
    outputXMSE, outputYMSE = MSENet(inL, inR)
    outputXMSE = torch.squeeze(outputXMSE)
    outputYMSE = torch.squeeze(outputYMSE)
    
    outputAziMSE = calcCircumAngle(50, 50, outputXMSE, outputYMSE)
    
    CosLosses[0,i] = CosLoss(outputAziMSE, labelAzi)
    MSELosses[0,i] = MSELoss(outputXMSE, labelX) + MSELoss(outputYMSE, labelY)
    
     
    # Compute MSE and Cos losses for angle network
    CosNet = CosNet.to(device)
    
    outputAziCos = CosNet(inL, inR)
    outputAziCos = torch.squeeze(outputAziCos)
    outputXCos, outputYCos = calcCircumPos(50, 0, 50, outputAziCos)
    
    CosLosses[1,i] = CosLoss(outputAziCos, labelAzi)
    MSELosses[1,i] = MSELoss(outputXCos, labelX) + MSELoss(outputYCos, labelY)

del MSENet, CosNet, inL, inR, labelX, labelY, labelAzi

np.save(DATA_PATH+"exp1_MSE_outcomes", MSELosses)
np.save(DATA_PATH+"exp1_Cos_outcomes", CosLosses)

In [ ]:
MSELosses_exp1 = np.load(DATA_PATH+"exp1_MSE_outcomes.npy")
CosLosses_exp1 = np.load(DATA_PATH+"exp1_Cos_outcomes.npy")

print("MSE        COS")
print("MSE Losses")
print(np.mean(MSELosses_exp1, axis = 1))
print(np.std(MSELosses_exp1, axis = 1))

print("Cos Losses")
print(np.mean(CosLosses_exp1, axis = 1))
print(np.std(CosLosses_exp1, axis = 1))

In [ ]:
################
# EXPERIMENT 2 #
################

MSELosses = np.zeros((len(DATASETS_EXP2), len(nets), EPOCHS))

for i,dataset in enumerate(DATASETS_EXP2):
    print("Dataset "+str(i))

    dataLoader = DataLoader(dataset, batch_size = BATCH_SIZE)
    
    for j,net in enumerate(nets):
        print("Network "+str(j))

        net.to(device)

        for k in range(EPOCHS):
            if k%400 == 0:
                print("Epoch "+str(k)+"/"+str(EPOCHS))
            
            inL, inR, labelX, labelY, _ = next(iter(dataLoader))
            inL = inL.double().to(device)
            inR = inR.double().to(device)
            labelX = labelX.double().to(device)
            labelY = labelY.double().to(device)

            outputX, outputY = net(inL, inR)
            outputX = torch.squeeze(outputX)
            outputY = torch.squeeze(outputY)

            MSELosses[i,j,k] = MSELoss(outputX, labelX) + MSELoss(outputY, labelY)

        del net, inL, inR, labelX, labelY

np.save(DATA_PATH+"exp2_outcomes", MSELosses)

In [ ]:
losses_exp2 = np.load(DATA_PATH+"exp2_outcomes.npy")

print("50    500    5000")
print("RAD = 50")
print(np.mean(losses_exp2[0], axis=1))
print(np.std(losses_exp2[0], axis=1))
print("RAD = 500")
print(np.mean(losses_exp2[1], axis=1))
print(np.std(losses_exp2[1], axis=1))
print("RAD = 5000")
print(np.mean(losses_exp2[2], axis=1))
print(np.std(losses_exp2[2], axis=1))

In [ ]:
################
# EXPERIMENT 3 #
################

BATCH_SIZE_EXP3 = 1
TEST_SIZE_EXP3 = 1500
EPOCHS_EXP3 = int(TEST_SIZE_EXP3/BATCH_SIZE_EXP3)
ITDs = np.zeros((len(DATASETS_EXP3), EPOCHS_EXP3))
IIDs = np.zeros((len(DATASETS_EXP3), EPOCHS_EXP3))
threshold = 2e-12
MSENet = ConvNet(MIN_LENGTH_MSECOS).double()
MSENet.load_state_dict(torch.load(MODELS_PATH+"ConvNet_MSELoss", map_location="cpu"))
MSENet.eval()
MSENet = MSENet.to(device)

for i,dataset in enumerate(DATASETS_EXP3):
    
    print("Dataset "+str(i))
    dataLoader = DataLoader(dataset, batch_size = BATCH_SIZE_EXP3)
    
    for j in range(EPOCHS_EXP3):
        
        ITDs_batch = []
        IIDs_batch = []
        
        if j%100 == 0:
                print("Epoch "+str(j)+"/"+str(EPOCHS_EXP3))
                
        inLs, inRs, _, _, _ = next(iter(dataLoader))
        
        for signalIdx in range(inLs.size(0)):
            inL = inLs[signalIdx].numpy()
            inR = inRs[signalIdx].numpy()

            inL[np.where(np.abs(inL) < threshold)] = 0
            inR[np.where(np.abs(inR) < threshold)] = 0

            ITDs_batch.append(computeMeanITD(inL, inR))
            IIDs_batch.append(computeMeanIID(inL, inR))
        
        ITDs[i,j] = np.mean(ITDs_batch)
        IIDs[i,j] = np.mean(IIDs_batch)
        
        inLs = inLs.double().to(device)
        inRs = inRs.double().to(device)
        _, _ = MSENet(inLs, inRs, record_acts = True)
        
        del inLs, inRs

del net        
        
import scipy.stats

np.save(DATA_PATH+"exp3_ITDs.npy", ITDs)
np.save(DATA_PATH+"exp3_IIDs.npy", IIDs)

In [ ]:
ITDs = np.load(DATA_PATH+"exp3_ITDs.npy")
IIDs = np.load(DATA_PATH+"exp3_IIDs.npy")

for i,freq in enumerate(["LowFreq","MedFreq","HigFreq"]):
    print(freq+" ITD correlation = " + str(scipy.stats.pearsonr(MSENet.fc_activations[1500*i:1500*(i+1)], ITDs[i])))
    print(freq+" IID correlation = " + str(scipy.stats.pearsonr(MSENet.fc_activations[1500*i:1500*(i+1)], IIDs[i])))

print(np.min(ITDs, axis=1))
print(np.max(ITDs, axis=1))

print(np.min(IIDs, axis=1))
print(np.max(IIDs, axis=1)) 